## Memory allocations

One of the most important things to understand when designing code that performs well is to understand how memory is allocated and deallocated.  The fallacy that garbage collected languages promise, is that "you don't have to worry about memory (de)allocation".  But this really isn't true.  GC'ed languages still pay a price:

- Easier memory leaks
- Poor performing code to to lots of memory copying
- Unsuitable to deterministic runtimes due to garbage collection pauses

System's program languages do not necessarily solve these problems with a silver bullet.  But they give you the tools to do so unlike GC'ed languages.

To give an example, let's see the performance of passing by reference, vs passing by copy


## Stack and heap

It's important to understand the stack and the heap when working with rust, and the earlier you understand this concept the better.

Many modern languages do not make a distinction between the stack and heap, so this may be a new concept.  The stack is a region that the Operating System uses to execute instructions of your program.  When you define a function, room has to be made on the stack for the arguments, and when the function exits, all the previously allocated elements on the stack are popped off and the return value is pushed onto the stack.

This is why recursive functions can `stack overflow`, because on each new function call, room for the arguments of the function are pushed onto the stack, and if this happens too much before the recursion stops, the stack space can be fully consumed (because the Operating System has a limit on how big the stack space is, and this is determined by the kernel when it is built usually)

So let's see how this works.

In [ ]:
use std::mem::{size_of, size_of_val};

fn printa<A>(x: &A, val: &str) {
    println!("Address of {val}: {:p}", x);
}

fn stack_print() {
    let a = 1;
    let b = 2;
    let c = "hi";
    let d = Box::new(10);
    
    printa(&a, "a");
    println!("Size of a is {}", size_of_val(&a));
    printa(&b, "b");
    println!("Size of b is {}", size_of_val(&b));
    printa(&c, "c");
    println!("Size of c is {}", size_of_val(&c));
    printa(&d, "d");
    println!("Size of d is {}", size_of_val(&d));
}

stack_print();

In [ ]:
struct Foo {}

let a1 = Foo {};
let a2 = Foo {};

printa(&a1, "a1");
printa(&a2, "a2");

Notice that both `a1` and `a2` point to the same address.  Also note we had to take a reference to these two values to be able to use the `:p` formatting in `println!`.

But why do they both point to the same address?  Didn't we create 2 new variables of type `Foo`?  Shouldn't we see two different addresses?

The answer is that unlike most garbage collected languages that create instances of data on the heap, `Foo {}` is just a value, no different 10, but, Foo also has no data associated with it.  What if we made a struct with data?

In [ ]:
#[derive(Debug)]
struct Bar { x: u32 }

let x = Bar { x : 10 };
let y = Bar { x: 20 };
printa(&x, "x");
printa(&y, "y");

Notice that now, we have different addresses for each.  This is because rust has to allocate memory for this data type unlike with Foo.

So far all these variables have lived on the stack.  What if we have a Box or Rc?  An important distinction must be made between the adress of a variable on the stack, with what it points to.  We will illustrate this with the following:

In [ ]:
let z = Box::new(y);
printa(&z, "z");
let raw: *mut Bar = Box::into_raw(z);
println!("Address in memory of z is {raw:p}");

Why did `z` print one address, but `raw` pointed to a different value?

In rust, we must differentiate between:

1. the name of the symbol, eg `z`
2. the value of the symbol
3. where the data that symbol refers to lives

Unfortunately, all 3 of these get easily confused, especially when we start talking about references or pointers (and a reference is basically a safe pointer).  When data just lives on the stack, 1 and 2 are typically a simple mapping.  For example with 

```
let x = 10;
```

then we can think of x **as** 10.  But technically, `x` is the name of the symbol, which points to some memory on the stack, and the value of 10.  When we run the `printa` function, we are showing the address on the stack.

In [ ]:
let x = 10;
println!("The address on the stack of x is {:p} and its value is {}", &x, *&x);

This explains why `z` (the symbol name that refers to some address on the stack) has a different address than the actual value of `z`.  This is because point of #3.  Since `z` value is actually a smart pointer, where the data that `z` refers to is different.

In [ ]:
let q = Bar { x: 30 };
let z = Box::new(q);
println!("The address on the stack of z is {:p} and its value is\n{:#?}", &z, *&z);
let raw = Box::into_raw(z);
unsafe {
    println!("The address of what z points to is {:p} and its value is\n{:#?}", raw, *raw);
}

Now that we have a basic understanding of the difference between the stack, let's look at how rust deals with &str and String.

So first off, why two different kinds of strings?  In rust, performance is important for systems programming so having control over stack or heap allocated data is important.  But more importantly, some data types we can't know at compile time, because they may grow in size.  For example, a Vec in rust is a type that can hold values of another but grow dynamically.  A String in rust, is a dynamically resizeable type that happens to hold u8 values, that happen to be UTF8 encoded.

In [ ]:
// name has type String, and name itself lives on the stack
// however, the data of name lives on the heap.  Notice how this is similiar to Box
let name = String::from("Sean");

printa(&name, "name");
let name_box = name.into_boxed_str();
let data_addr_of_name = Box::into_raw(name_box);
unsafe {
    println!("The address of what name points to is {:p}", data_addr_of_name);
}

let sean = "Sean";
printa(&sean, "sean");
let str_size = std::mem::size_of_val(sean);
println!("Size of sean is {str_size}");

let full = "sean toner";
let full_size = std::mem::size_of_val(full);
println!("Size of full is {full_size}");
let full_len = full.len();
println!("Size of full is {full_len}");

let name = String::from("Sean Toner");
println("size of name is {}", )
